For Week 4 I have to answer to questions below

#### 1) A description of the problem and a discussion of the background. (15 marks)

#### 2) A description of the data and how it will be used to solve the problem. (15 marks)

Okay let's start with the first question

#### 1) A Description of the problem and a discussion of the background

"Which store,shop or restaurant in Baltimore, MD, USA has the most incident?"

"Show some basic statistics, what type of crime happened by the type of store?

"Visualize Top 5 dangerous store,shop or restaurant in each neighborhood by utilizing Foursquare data and baltimore crime data"



#### 2) A description of the data and how it will be used to solve the problem

Baltimore Zipcode: http://ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=md&StName=Georgia&StFIPS=13&FIPS=24510

Baltimore Crime Data : https://data.baltimorecity.gov/datasets/part1-crime-data/explore?location=39.300436%2C-76.606438%2C11.36&showTable=true

## We are ready to start the project. The project compose with 4 steps
### 0) Import all the neccessary libraries
### 1) Play around and clean "Baltimore Crime Data"
### 2) Play around and clean "Baltimore FourSquare Data"
### 3) Combine two data
### 4) Visualize with folium


In [199]:
import numpy as np
import pandas as pd
import csv
import requests
import folium
import json
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
print('Libraries imported')

Libraries imported


Awesome step 0 done. Start indicates half done.

# Step 1) Play around and clean "Baltimore Crime Data"

In [200]:
# Bring CSV file and make a data frame out of it
crime_df = pd.read_csv (r'/Users/gihonglee/Downloads/Part1_Crime_data.csv')
crime_df.head(5)

,X,Y,RowID,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,VRIName,Total_Incidents,Shape
0,1.424294e+06,599933.502296,1,2021/07/31 09:06:21+00,1A,2000 KENNEDY AVE,HOMICIDE,Inside,FIREARM,312,EASTERN,EAST BALTIMORE MIDWAY,39.3133,-76.6043,"(39.3133,-76.6043)",VACANT DWELLING,Eastern 1,1,NaN
1,1.422335e+06,601637.004732,2,2021/07/31 14:30:00+00,6L,400 E 25TH ST,LARCENY,Inside,NaN,513,NORTHERN,BARCLAY,39.3180,-76.6112,"(39.318,-76.6112)",STREET,NaN,1,NaN
2,1.402984e+06,609174.437883,3,2021/07/31 21:00:00+00,6D,3800 DOLFIELD AVE,LARCENY FROM AUTO,NaN,NaN,625,NORTHWEST,DOLFIELD,39.3389,-76.6795,"(39.3389,-76.6795)",NaN,NaN,1,NaN
3,1.424228e+06,569446.823751,4,2021/07/31 01:00:00+00,6D,4300 5TH ST,LARCENY FROM AUTO,Inside,NaN,913,SOUTHERN,BROOKLYN,39.2296,-76.6050,"(39.2296,-76.605)",STREET,NaN,1,NaN
4,1.413008e+06,599194.993195,5,2021/07/31 12:30:00+00,4E,2600 PENNSYLVANIA AVE,COMMON ASSAULT,Outside,NaN,733,WESTERN,PENN NORTH,39.3114,-76.6442,"(39.3114,-76.6442)",OTHER/RESIDENTIAL,Western,1,NaN


Let me clean the data

In [201]:
crime_df = crime_df.drop(columns = ['X','Y','RowID','CrimeCode','Weapon','Post','GeoLocation','Premise','VRIName','Total_Incidents','Shape'])

In [202]:
crime_df.head(5)

,CrimeDateTime,Location,Description,Inside_Outside,District,Neighborhood,Latitude,Longitude
0,2021/07/31 09:06:21+00,2000 KENNEDY AVE,HOMICIDE,Inside,EASTERN,EAST BALTIMORE MIDWAY,39.3133,-76.6043
1,2021/07/31 14:30:00+00,400 E 25TH ST,LARCENY,Inside,NORTHERN,BARCLAY,39.3180,-76.6112
2,2021/07/31 21:00:00+00,3800 DOLFIELD AVE,LARCENY FROM AUTO,NaN,NORTHWEST,DOLFIELD,39.3389,-76.6795
3,2021/07/31 01:00:00+00,4300 5TH ST,LARCENY FROM AUTO,Inside,SOUTHERN,BROOKLYN,39.2296,-76.6050
4,2021/07/31 12:30:00+00,2600 PENNSYLVANIA AVE,COMMON ASSAULT,Outside,WESTERN,PENN NORTH,39.3114,-76.6442


In [203]:
# Clean the crimeDateTime
crime_df["CrimeDateTime"] = crime_df["CrimeDateTime"].astype(str).str[0:10]

Drop na rows

In [204]:
print(crime_df.shape)
crime_df = crime_df.dropna()
print(crime_df.shape)
print("Data has been cleaned")

(343727, 8)
(296487, 8)
Data has been cleaned


In [205]:
crime_df.head(5)

,CrimeDateTime,Location,Description,Inside_Outside,District,Neighborhood,Latitude,Longitude
0,2021/07/31,2000 KENNEDY AVE,HOMICIDE,Inside,EASTERN,EAST BALTIMORE MIDWAY,39.3133,-76.6043
1,2021/07/31,400 E 25TH ST,LARCENY,Inside,NORTHERN,BARCLAY,39.3180,-76.6112
3,2021/07/31,4300 5TH ST,LARCENY FROM AUTO,Inside,SOUTHERN,BROOKLYN,39.2296,-76.6050
4,2021/07/31,2600 PENNSYLVANIA AVE,COMMON ASSAULT,Outside,WESTERN,PENN NORTH,39.3114,-76.6442
5,2021/07/31,2700 RAYNER AVE,COMMON ASSAULT,Outside,WESTERN,MOSHER,39.2964,-76.6616


#### Let's manaually create the latitude and longitude of neighborhood by getting the mean value of lat and lng

In [206]:
hood_coor = crime_df.groupby(by = "Neighborhood").mean()
hood_coor.reset_index(inplace = True)
hood_coor[hood_coor["Neighborhood"] == "BROOKLYN"]

,Neighborhood,Latitude,Longitude
26,BROOKLYN,39.233851,-76.603065


# 2) Play around and clean "Baltimore FourSquare Data"

#### Define Foursquare Credentials and Version

In [207]:
CLIENT_ID = 'GHM3DHK0SNYALRT1CZTQOFEU4BRENGUSVM451QCG4E2JFDVS' # your Foursquare ID
CLIENT_SECRET = 'X2PMXTL3HGU1JQXJVJKCKXCLITPC4024TFPUTXNODZV25Z5S' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GHM3DHK0SNYALRT1CZTQOFEU4BRENGUSVM451QCG4E2JFDVS
CLIENT_SECRET:X2PMXTL3HGU1JQXJVJKCKXCLITPC4024TFPUTXNODZV25Z5S


First of all let's try to get top 100 venues from a location.
For the convience, let's pick a location to be Downtown

In [208]:
lat = 39.2895
lng = -76.6151

In [209]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat,
    lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GHM3DHK0SNYALRT1CZTQOFEU4BRENGUSVM451QCG4E2JFDVS&client_secret=X2PMXTL3HGU1JQXJVJKCKXCLITPC4024TFPUTXNODZV25Z5S&v=20180604&ll=39.2895,-76.6151&radius=500&limit=100'

Send the GET request and examine the resutls

In [210]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '610c07c8e3778f09662d50de'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown Baltimore',
  'headerFullLocation': 'Downtown Baltimore, Baltimore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 72,
  'suggestedBounds': {'ne': {'lat': 39.2940000045, 'lng': -76.60929656813734},
   'sw': {'lat': 39.284999995499994, 'lng': -76.62090343186266}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a9c2c73f964a520453620e3',
       'name': 'Kimpton Hotel Monaco Baltimore Inner Harbor',
       'location': {'address': '2 N Charles St',
        'crossStreet': 'Baltimore St.',
        'lat': 39.2

In [212]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Let's create a function to repeat the same process to all the neighborhoods in Baltimore

In [292]:
def getNearbyVenues(names, lats, lngs, radius = 500):
    
    nearby_venues_columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Address',
                            'Venue Category']
    nearby_venues = pd.DataFrame(columns = nearby_venues_columns)
    filtered_columns = ['venue.name','venue.location.lat','venue.location.lng','venue.location.address','venue.categories']

    for name, lat, lng in zip(names, lats, lngs):
        print(name)       
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        nearby_venues_table = json_normalize(results)
        if set(filtered_columns).issubset(set(nearby_venues_table.columns)):
            nearby_venues_table = nearby_venues_table[filtered_columns]
            nearby_venues_table['venue.categories'] = nearby_venues_table.apply(get_category_type, axis = 1)
            nearby_venues_table.insert(loc = 0, column = "lng", value = lng)
            nearby_venues_table.insert(loc = 0, column = "lat", value = lat)
            nearby_venues_table.insert(loc = 0, column = "name", value = name)
            nearby_venues_table.rename(columns = dict(zip(nearby_venues_table.columns, nearby_venues_columns)),inplace = True)
            print(nearby_venues_table.shape)
            print(nearby_venues.shape)
            nearby_venues = nearby_venues.append(nearby_venues_table)
            #print(nearby_venues.shape)
    
        # return only relevant information for each nearby venue
        
#         venues_list.append([(
#             name,
#             lat,
#             lng,
#             v['venue']['name'],
#             v['venue']['location']['lat'],
#             v['venue']['location']['lng'],
#             v['venue']['location']['address'],
#             v['venue']['categories'][0]['name']) for v in results])
  
    
    

    return(nearby_venues)


In [293]:
a = getNearbyVenues(hood_coor["Neighborhood"], hood_coor["Latitude"], hood_coor["Longitude"], radius = 500)


ABELL
(46, 8)
(0, 8)
ALLENDALE
(1, 8)
(46, 8)
ARCADIA
(2, 8)
(47, 8)
ARLINGTON
(14, 8)
(49, 8)
ARMISTEAD GARDENS
(1, 8)
(63, 8)
ASHBURTON
BALTIMORE HIGHLANDS
(31, 8)
(64, 8)
BARCLAY
(21, 8)
(95, 8)
BARRE CIRCLE
(31, 8)
(116, 8)
BAYVIEW
(14, 8)
(147, 8)
BEECHFIELD
(4, 8)
(161, 8)
BELAIR-EDISON
(8, 8)
(165, 8)
BELAIR-PARKSIDE
(5, 8)
(173, 8)
BELLONA-GITTINGS
(3, 8)
(178, 8)
BELVEDERE
(28, 8)
(181, 8)
BEREA
(5, 8)
(209, 8)
BETTER WAVERLY
(18, 8)
(214, 8)
BEVERLY HILLS
(13, 8)
(232, 8)
BIDDLE STREET
(5, 8)
(245, 8)
BLYTHEWOOD
(7, 8)
(250, 8)
BOLTON HILL
(12, 8)
(257, 8)
BOYD-BOOTH
(11, 8)
(269, 8)
BREWERS HILL
(32, 8)
(280, 8)
BRIDGEVIEW/GREENLAWN
(3, 8)
(312, 8)
BROADWAY EAST
BROENING MANOR
(9, 8)
(315, 8)
BROOKLYN
(8, 8)
(324, 8)
BURLEITH-LEIGHTON
(18, 8)
(332, 8)
BUTCHER'S HILL
(9, 8)
(350, 8)
CALLAWAY-GARRISON
(3, 8)
(359, 8)
CAMERON VILLAGE
(4, 8)
(362, 8)
CANTON
(59, 8)
(366, 8)
CANTON INDUSTRIAL AREA
(37, 8)
(425, 8)
CARE
(19, 8)
(462, 8)
CARROLL - CAMDEN INDUSTRIAL AREA
(31, 8)
(48

(7, 8)
(3752, 8)
WALTHERSON
(3, 8)
(3759, 8)
WASHINGTON HILL
(17, 8)
(3762, 8)
WASHINGTON VILLAGE/PIGTOWN
(22, 8)
(3779, 8)
WAVERLY
(9, 8)
(3801, 8)
WEST ARLINGTON
(2, 8)
(3810, 8)
WEST FOREST PARK
(1, 8)
(3812, 8)
WEST HILLS
(2, 8)
(3813, 8)
WESTFIELD
(7, 8)
(3815, 8)
WESTGATE
(4, 8)
(3822, 8)
WESTPORT
(3, 8)
(3826, 8)
WILHELM PARK
(3, 8)
(3829, 8)
WILSON PARK
(4, 8)
(3832, 8)
WINCHESTER
WINDSOR HILLS
(1, 8)
(3836, 8)
WINSTON-GOVANS
(13, 8)
(3837, 8)
WOODBERRY
(9, 8)
(3850, 8)
WOODBOURNE HEIGHTS
(11, 8)
(3859, 8)
WOODBOURNE-MCCABE
(9, 8)
(3870, 8)
WOODMERE
(20, 8)
(3879, 8)
WRENLANE
(5, 8)
(3899, 8)
WYMAN PARK
(53, 8)
(3904, 8)
WYNDHURST
(10, 8)
(3957, 8)
YALE HEIGHTS
(4, 8)
(3967, 8)
YORK-HOMELAND
(9, 8)
(3971, 8)


#### Finally we got the baltimore store data!! Part2 Done!

In [297]:
baltimore_store = a
baltimore_store

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Address,Venue Category
0,ABELL,39.326032,-76.610699,32nd Street Farmer's Market,39.327362,-76.610851,400 E 32nd St,Farmers Market
1,ABELL,39.326032,-76.610699,Pete's Grille,39.327064,-76.609593,3130 Greenmount Ave,Breakfast Spot
2,ABELL,39.326032,-76.610699,The Book Thing,39.325253,-76.610272,3001 Vineyard Ln,Bookstore
3,ABELL,39.326032,-76.610699,Normal's Books & Records,39.326012,-76.609903,425 E 31st St,Record Shop
4,ABELL,39.326032,-76.610699,Peabody Heights Brewery,39.324744,-76.610819,401 E 30th St,Brewery
...,...,...,...,...,...,...,...,...
4,YORK-HOMELAND,39.353207,-76.610456,Popeyes Louisiana Kitchen,39.349473,-76.610067,5002 York Rd,Fried Chicken Joint
5,YORK-HOMELAND,39.353207,-76.610456,CVS pharmacy,39.352111,-76.610336,5200 York Rd,Pharmacy
6,YORK-HOMELAND,39.353207,-76.610456,King Wok,39.351083,-76.609429,5105 York Rd,Chinese Restaurant
7,YORK-HOMELAND,39.353207,-76.610456,Corky's Liquors,39.355779,-76.610051,5404 York Rd,Liquor Store


## Part 3) Let's try to Combine two data
#### Before combining let's recap what data we have

In [302]:
# Baltimore Crime Data
crime_df.head(3)
# Baltimore Store Data
#baltimore_store.head(3)

,CrimeDateTime,Location,Description,Inside_Outside,District,Neighborhood,Latitude,Longitude
0,2021/07/31,2000 KENNEDY AVE,HOMICIDE,Inside,EASTERN,EAST BALTIMORE MIDWAY,39.3133,-76.6043
1,2021/07/31,400 E 25TH ST,LARCENY,Inside,NORTHERN,BARCLAY,39.3180,-76.6112
3,2021/07/31,4300 5TH ST,LARCENY FROM AUTO,Inside,SOUTHERN,BROOKLYN,39.2296,-76.6050


#### we can either compare by street name which is -> 1) crime_df[Location] & baltimore_store[Venue] 
#### or we can compare by the coordinate of the location. 
#### Either way, we need to clean the data. So let's
#### step 1) clean the data for the location
#### step 2) see how many overlap is there between crime and store data

In [310]:
# so first let's make all upper letter for the venue address of store data
baltimore_store["Venue Address"] = baltimore_store["Venue Address"].str.upper()
baltimore_store["Location"] = baltimore_store["Venue Address"]

In [318]:
baltimore_store["Venue Address"]
print(crime_df.shape)
merge_df = crime_df.merge(baltimore_store, how='inner', on='Location')
print(merge_df.shape)

(296487, 8)
(31355, 16)


It is interesting out of 296,487 crime cases, 31,355 cases were happend either on store or outside of the store

However, before going to the further analysis, let's clean the data a little bit.


In [325]:
merge_df.tail(5)

,CrimeDateTime,Location,Description,Inside_Outside,District,Neighborhood_x,Latitude,Longitude,Neighborhood_y,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Address,Venue Category
31350,2019/12/24,500 W COLD SPRING LN,BURGLARY,I,NORTHERN,KESWICK,39.3445,-76.6311,EVERGREEN,39.345547,-76.628235,ROLAND PARK BAGEL CO.,39.344612,-76.631178,500 W COLD SPRING LN,Bagel Shop
31351,2019/12/24,500 W COLD SPRING LN,BURGLARY,I,NORTHERN,KESWICK,39.3445,-76.6311,KESWICK,39.343444,-76.630594,ROLAND PARK BAGEL CO.,39.344612,-76.631178,500 W COLD SPRING LN,Bagel Shop
31352,2019/12/24,500 W COLD SPRING LN,BURGLARY,I,NORTHERN,KESWICK,39.3445,-76.6311,ROLAND PARK,39.345305,-76.634957,ROLAND PARK BAGEL CO.,39.344612,-76.631178,500 W COLD SPRING LN,Bagel Shop
31353,2020/07/31,REISTERSTOWN RD,BURGLARY,I,NORTHWEST,REISTERSTOWN STATION,39.3563,-76.7028,LUCILLE PARK,39.340572,-76.675024,VALLEY VILLAGE WINE AND SPIRITS,39.339379,-76.680155,REISTERSTOWN RD,Liquor Store
31354,2020/05/31,400 W COLD SPRING LN,BURGLARY,I,NORTHERN,KESWICK,39.3446,-76.6305,KESWICK,39.343444,-76.630594,VIDEO AMERICAIN - ROLAND PARK,39.343859,-76.635222,400 W COLD SPRING LN,Video Store


In [328]:
merge_col = ["CrimeDateTime", "Location", "Description", "Inside_Outside", "Latitude", "Longitude","Venue","Venue Category"]
merge_df = merge_df[merge_col]
merge_df

,CrimeDateTime,Location,Description,Inside_Outside,Latitude,Longitude,Venue,Venue Category
0,2021/07/31,5800 EASTERN AVE,AGG. ASSAULT,Inside,39.2875,-76.5464,LA BAMBA LIQUORS,Mexican Restaurant
1,2018/08/19,5800 EASTERN AVE,COMMON ASSAULT,I,39.2875,-76.5464,LA BAMBA LIQUORS,Mexican Restaurant
2,2018/07/28,5800 EASTERN AVE,COMMON ASSAULT,O,39.2875,-76.5461,LA BAMBA LIQUORS,Mexican Restaurant
3,2018/06/18,5800 EASTERN AVE,COMMON ASSAULT,O,39.2875,-76.5464,LA BAMBA LIQUORS,Mexican Restaurant
4,2017/08/26,5800 EASTERN AVE,LARCENY FROM AUTO,O,39.2875,-76.5464,LA BAMBA LIQUORS,Mexican Restaurant
...,...,...,...,...,...,...,...,...
31350,2019/12/24,500 W COLD SPRING LN,BURGLARY,I,39.3445,-76.6311,ROLAND PARK BAGEL CO.,Bagel Shop
31351,2019/12/24,500 W COLD SPRING LN,BURGLARY,I,39.3445,-76.6311,ROLAND PARK BAGEL CO.,Bagel Shop
31352,2019/12/24,500 W COLD SPRING LN,BURGLARY,I,39.3445,-76.6311,ROLAND PARK BAGEL CO.,Bagel Shop
31353,2020/07/31,REISTERSTOWN RD,BURGLARY,I,39.3563,-76.7028,VALLEY VILLAGE WINE AND SPIRITS,Liquor Store


In [343]:
print(merge_df.Inside_Outside.unique())
merge_df.replace({"Inside" : "I", "Outside" : "O"}, inplace = True)
merge_df.groupby(by = "Inside_Outside").count()

['I' 'O']


,CrimeDateTime,Location,Description,Latitude,Longitude,Venue,Venue Category
Inside_Outside,,,,,,,
I,19824,19824,19824,19824,19824,19824,19824
O,11531,11531,11531,11531,11531,11531,11531


In [345]:
=19824/(19824+11531)

0.6322436612980386

#### Around 63% percent of crime happened inside of the store and 37% of the crime happend outside of the store.

Now let's see 1) what type of store has been involved in crime
2) Top 2 notorious store in the baltimore


In [348]:
#1) what type of store is fragile to crime

In [354]:
merge_df.groupby(by = "Venue Category").count().sort_values(by = "CrimeDateTime", ascending = False).head(10)

,CrimeDateTime,Location,Description,Inside_Outside,Latitude,Longitude,Venue
Venue Category,,,,,,,
Pharmacy,2779,2779,2779,2779,2779,2779,2779
Bus Station,2548,2548,2548,2548,2548,2548,2548
Metro Station,1929,1929,1929,1929,1929,1929,1929
Coffee Shop,1509,1509,1509,1509,1509,1509,1509
Grocery Store,1224,1224,1224,1224,1224,1224,1224
Hotel,1041,1041,1041,1041,1041,1041,1041
Seafood Restaurant,810,810,810,810,810,810,810
Convenience Store,794,794,794,794,794,794,794
Supplement Shop,684,684,684,684,684,684,684


Not quite sure why Pharmacy is the place that crime happend the most. let's take a deeper look into it.

In [360]:
phar_df = merge_df[merge_df["Venue Category"] == "Pharmacy"]
phar_df.groupby(by = "Description").count().sort_values(by = "Location")

,CrimeDateTime,Location,Inside_Outside,Latitude,Longitude,Venue,Venue Category
Description,,,,,,,
ROBBERY - CARJACKING,3,3,3,3,3,3,3
ROBBERY - RESIDENCE,10,10,10,10,10,10,10
RAPE,17,17,17,17,17,17,17
SHOOTING,46,46,46,46,46,46,46
AUTO THEFT,77,77,77,77,77,77,77
LARCENY FROM AUTO,98,98,98,98,98,98,98
ROBBERY - STREET,115,115,115,115,115,115,115
BURGLARY,152,152,152,152,152,152,152
ROBBERY - COMMERCIAL,160,160,160,160,160,160,160


Aha I guess due to running 24hours, the crime happens often at night. In order to prove this I have to get the time data from the original and do more analysis but,,,, let's move on for now

#### Now let's see which store is the "Notorious store"

In [473]:
c_df = merge_df.groupby(by = "Location").count().sort_values(by = "Venue", ascending = False)

c_df["percent"] = (c_df['CrimeDateTime'] / c_df['CrimeDateTime'].sum()) * 100
c_df.reset_index(inplace = True)
final_df = pd.merge(c_df, merge_df.groupby("Location",as_index=False).first(), left_on='Location', right_on='Location', how='left')
columns = ["Location", "CrimeDateTime_x","Latitude_y", "Longitude_y","Venue_y","Venue Category_y" ,"Description_y"]
final_df = final_df[columns]
new_col = ["Location", "Num_Crime","Lat", "Lng","Venue","Category","Desc"]
final_df.rename(columns=dict(zip(columns,new_col)), inplace = True)



## 4) Visualize with folium 

In [474]:
final_df.head()
final_df["Num_Crime"].mean()
final_df

,Location,Num_Crime,Lat,Lng,Venue,Category,Desc
0,2400 LIBERTY HEIGHTS AVE,4459,39.3186,-76.6540,MONDAWMIN METRO STATION,Metro Station,LARCENY
1,200 E PRATT ST,3354,39.2866,-76.6121,BATH & BODY WORKS,Cosmetics Shop,LARCENY
2,400 E PRATT ST,1415,39.2867,-76.6099,SHAKE SHACK,Burger Joint,LARCENY
3,300 LIGHT ST,1116,39.2851,-76.6131,HYATT REGENCY BALTIMORE INNER HARBOR,Hotel,LARCENY
4,5200 YORK RD,995,39.3540,-76.6098,CVS PHARMACY,Pharmacy,LARCENY
...,...,...,...,...,...,...,...
201,PATAPSCO AVE,1,39.2363,-76.6023,NEW YORK FRIED CHICKEN,Fast Food Restaurant,COMMON ASSAULT
202,REISTERSTOWN RD,1,39.3563,-76.7028,VALLEY VILLAGE WINE AND SPIRITS,Liquor Store,BURGLARY
203,SARATOGA ST & HOLLIDAY ST,1,39.2925,-76.6104,ZEKE'S COFFEE @ BALTIMORE FARMER'S MARKET,Coffee Shop,LARCENY
204,W BELVEDERE AVE,1,39.3524,-76.6692,BELVEDERE & LANIER,Intersection,AUTO THEFT


In [486]:
# Now it is the time to do some visualization! 
# I want to do it so that the latitude with the high crime to have a bigger circle and low crime to have smaller circle
# first of all let's set the latitude and longitude, for the convience, we will just go by the mean of final df
lat = final_df["Lat"].mean()
lng = final_df["Lng"].mean()

# create map
map_clusters = folium.Map(location = [lat, lng], zoom_start = 11)
# set color scheme for the clusters

#x = np.arange(kclusters)
#ys = [i +x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0,1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the maps
#markers_colors = []
for lat, lon, ven, Desc, mag in zip(final_df['Lat'], final_df['Lng'],\
                                 final_df['Venue'],final_df['Desc'], final_df['Num_Crime']):
    label = folium.Popup(ven + " - " + Desc , parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = (mag+300)/300,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'red',
        fill_opacity = 0.7).add_to(map_clusters)
map_clusters